In [6]:
#!cp 2-identity_block.py 3-projection_block.py

In [2]:
import tensorflow.keras as K

In [ ]:
#!/usr/bin/env python3

import tensorflow.keras as K
#inception_block = __import__('0-inception_block').inception_block
def inception_block(A_prev, filters):
    """ doc """
    MaxPooling2D = K.layers.MaxPooling2D
    Conv2D = K.layers.Conv2D
    Concatenate = K.layers.Concatenate()
    
    F1, F3R, F3, F5R, F5, FPP = filters
    
    layer1x1_0 = Conv2D(F1, 1, activation='relu')(A_prev) #
    
    layer1x1_1 = Conv2D(F3R, 1, padding='same', activation='relu')(A_prev)
    
    layer3x3 = Conv2D(F3, 3, padding='same', activation='relu')(layer1x1_1) #
    
    layer1x1_2 = Conv2D(F5R, 1, padding='same', activation='relu')(A_prev)
    
    layer5x5 = Conv2D(F5, 5, padding='same',  activation='relu')(layer1x1_2) #
    
    layerMax = MaxPooling2D(1)(A_prev)
    
    layer1x1_3 = Conv2D(FPP, 1, padding='same', activation='relu')(layerMax) #
    
    layer_out = Concatenate([layer1x1_0, layer3x3, layer5x5, layer1x1_3])
    return layer_out
if __name__ == '__main__':
    X = K.Input(shape=(224, 224, 3))
    Y = inception_block(X, [64, 96, 128, 16, 32, 32])
    model = K.models.Model(inputs=X, outputs=Y)
    model.summary()

In [ ]:
 [(None, 224, 224, 64), (None, 222, 222, 128), (None, 220, 220, 32), (None, 224, 224, 32)]

In [ ]:
# one by one from the table
def inception_network():
    """ doc """
    MaxPooling2D = K.layers.MaxPooling2D
    AveragePooling2D = K.layers.AveragePooling2D
    Conv2D = K.layers.Conv2D
    Dropout = K.layers.Dropout(rate=0.6)
    Dense = K.layers.Dense
    X = K.Input(shape=(224, 224, 3))
    
    layer7x7 = Conv2D(64, 7, 2, padding='same',  activation='relu')(X)
    layerMax = MaxPooling2D(3, 2, padding='same')(layer7x7)
    
    layer3x3 = Conv2D(64, 1, 1,  activation='relu')(layerMax)
    layer3x3 = Conv2D(192, 3, 1, padding='same',  activation='relu')(layer3x3)
    layerMax_2 = MaxPooling2D(3, 2, padding='same')(layer3x3)

    inception = inception_block(layerMax_2, [64, 96, 128, 16, 32, 32])
    inception = inception_block(inception, [128, 128, 192, 32, 96, 64])
    
    layerMax_2 = MaxPooling2D(3, 2, padding='same')(inception)
    inception = inception_block(layerMax_2, [192, 96, 208, 16, 48, 64])
    inception = inception_block(inception, [160, 112, 224, 24, 64, 64])
    inception = inception_block(inception, [128, 128, 256, 24, 64, 64])
    inception = inception_block(inception, [112, 144, 288, 32, 64, 64])
    inception = inception_block(inception, [256, 160, 320, 32, 128, 128])
    layerMax_2 = MaxPooling2D(3, 2, padding='same')(inception)
    inception = inception_block(layerMax_2, [256, 160, 320, 32, 128, 128])
    inception = inception_block(inception, [384, 192, 384, 48, 128, 128])
    layerAVG_2 = AveragePooling2D(7, 1)(inception)
    dropped = Dropout(layerAVG_2)
    # dropped = Dense(1000, activation="relu")(dropped)
    Y = Dense(1000, activation="softmax")(dropped)

    model = K.Model(inputs=X, outputs=Y)
    return model

if __name__ == '__main__':
    model = inception_network()
    model.summary()

In [5]:
def identity_block(A_prev, filters):
    """ doc """
    Conv2D = K.layers.Conv2D
    BatchNorm = K.layers.BatchNormalization
    Activation = K.layers.Activation
    Add = K.layers.Add
    F11, F3, F12 = filters
    layer1x1 = Conv2D(F11, 1, padding='same', kernel_initializer='he_normal')(A_prev)
    layer1x1 = BatchNorm()(layer1x1)
    layer1x1 = Activation('relu')(layer1x1)

    layer3x3 = Conv2D(F3, 3, padding='same', kernel_initializer='he_normal')(layer1x1)
    layer3x3 = BatchNorm()(layer3x3)
    layer3x3 = Activation('relu')(layer3x3)

    layer1x1 = Conv2D(F12, 1, padding='same', kernel_initializer='he_normal')(layer3x3)
    layer1x1 = BatchNorm()(layer1x1)

    layer_out = Add()([layer1x1, A_prev])
    layer_out = Activation('relu')(layer_out)

    return layer_out

if __name__ == '__main__':
    X = K.Input(shape=(224, 224, 256))
    Y = identity_block(X, [64, 64, 256])
    model = K.models.Model(inputs=X, outputs=Y)
    model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            (None, 224, 224, 256 0                                            
__________________________________________________________________________________________________
conv2d_6 (Conv2D)               (None, 224, 224, 64) 16448       input_3[0][0]                    
__________________________________________________________________________________________________
batch_normalization_6 (BatchNor (None, 224, 224, 64) 256         conv2d_6[0][0]                   
__________________________________________________________________________________________________
activation_6 (Activation)       (None, 224, 224, 64) 0           batch_normalization_6[0][0]      
__________________________________________________________________________________________________
conv2d_7 (

In [25]:
projection_block = __import__('3-projection_block').projection_block

def projection_block(A_prev, filters, s=2):
    """ doc """
    Conv2D = K.layers.Conv2D
    BatchNorm = K.layers.BatchNormalization
    Activation = K.layers.Activation
    Add = K.layers.Add
    F11, F3, F12 = filters
    layer1x1_s = Conv2D(F12, 1, 2,
                      kernel_initializer='he_normal')(A_prev)
    layer1x1_s = BatchNorm()(layer1x1_s)  #

    layer1x1 = Conv2D(F11, 1, 
                      kernel_initializer='he_normal')(A_prev)
    layer1x1 = BatchNorm()(layer1x1)
    layer1x1 = Activation('relu')(layer1x1)

    layer3x3 = Conv2D(F3, 3, padding='same',
                      kernel_initializer='he_normal')(layer1x1)
    layer3x3 = BatchNorm()(layer3x3)
    layer3x3 = Activation('relu')(layer3x3)

    layer1x1 = Conv2D(F12, 1, s,
                      kernel_initializer='he_normal')(layer3x3)
    layer1x1 = BatchNorm()(layer1x1)  #

    layer_out = Add()([layer1x1, layer1x1_s])
    layer_out = Activation('relu')(layer_out)

    return layer_out
if __name__ == '__main__':
    X = K.Input(shape=(224, 224, 3))
    Y = projection_block(X, [64, 64, 256])
    model = K.models.Model(inputs=X, outputs=Y)
    model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_23 (InputLayer)           (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
conv2d_89 (Conv2D)              (None, 224, 224, 64) 256         input_23[0][0]                   
__________________________________________________________________________________________________
batch_normalization_89 (BatchNo (None, 224, 224, 64) 256         conv2d_89[0][0]                  
__________________________________________________________________________________________________
activation_58 (Activation)      (None, 224, 224, 64) 0           batch_normalization_89[0][0]     
__________________________________________________________________________________________________
conv2d_90 